In [56]:
import torch
import numpy as np
import matplotlib.pyplot as plt

!pip install hmmlearn
from hmmlearn import hmm
from torch.distributions import uniform

import sys
sys.path.append("../")

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

import os
import argparse
import pickle
import timeit

from torch._utils import _accumulate
import torch.utils.data as data_utils
from torch.utils.data import Subset
import torch.nn.utils.clip_grad as clip_grad
import torch.optim.lr_scheduler as lr_scheduler

import copy

In [57]:
#Generate Datasets using HMM

def random_transmat(n_states):
    matrix = np.random.rand(n_states, n_states)
    return matrix/matrix.sum(axis=1)[:,None]

def random_startprob(n_states):
    startprob = np.random.rand(n_states)
    return startprob/startprob.sum()

def random_means(n_features):
    return np.random.randint(5, size=(n_features,n_features))

def generate_hmm(n_states, n_features , n_samples, length):
    #GENERATING A MODEL
    model = hmm.GaussianHMM(n_components=n_states, covariance_type="full")
    model.startprob_ = random_startprob(n_states)
    model.transmat_ = random_transmat(n_states)

    model.means_ = random_means(n_features)
    model.covars_ = np.tile(np.identity(n_features), (n_features, 1, 1))


    #SAMPLING FROM MODEL and STORING IN TENSOR

    #Number of Samples in Dataset
    dataset=[]
    states = []

    for i in range(n_samples):
        X, Z = model.sample(length)
        dataset.append(np.array(X))
        states.append(Z)

    dataset = np.stack(dataset)
    
    return dataset, np.array(states)

def generate_time_dependent_flip(n_samples, length, startprob, transmat):
    #GENERATING A MODEL


    model = hmm.GaussianHMM(n_components=n_states, covariance_type="full")
    model.startprob_ = startprob
    model.transmat_ = transmat

    #this doesn't actually matter for us
    model.means_ = np.array([[0.0, 0.0], 
                             [5.0, 10.0]])
    model.covars_ = np.tile(np.identity(2), (3, 1, 1))


    #SAMPLING FROM MODEL and STORING IN TENSOR
    
    #Number of Samples in Dataset
    dataset=[]

    for i in range(n_samples):
        X, Z = model.sample(length)
        dataset.append(np.array(Z))

    dataset = np.stack(dataset)
    
    return dataset

In [58]:
#Injecting Noise into Labels

#Given a flip_mask, flip an input
def flip(array, flip_mask):
    flipped_array = np.logical_xor(array, flip_mask, out=array)
    return flipped_array

#Class Independent / Time Independent
def flip_labels_basic(array, flip_probability):
    flip_mask = np.random.binomial(1, 0.5, len(array))
    return flip(array, flip_mask)

#Class Dependent / Time Independent
def flip_labels_class(array, flip_probability_0, flip_probability_1):
    flip_mask = []
    for elem in array:
        if elem == 0:
            to_flip = np.random.binomial(1, flip_probability_0, 1)[0]
            flip_mask.append(to_flip)
        else:
            to_flip = np.random.binomial(1, flip_probability_1, 1)[0]
            flip_mask.append(to_flip)
            
    return flip(array, flip_mask)

#Class Independent / Time Dependent
def flip_labels_time(array, startprob, transmat):
    flip_mask = generate_time_dependent_flip(1, len(array), startprob, transmat)[0]

    return flip(array, flip_mask)


#Class Dependent / Time Dependent
#This can be achieved by careful design of the transition matrix (transmat)

In [59]:
dataset,Z = generate_hmm(2,3,10, 100)

In [60]:
Z.shape

(10, 100)

In [61]:
startprob = random_startprob(2)
transmat = np.array([[0.95, 0.05],
                    [0.95, 0.05]])

In [62]:
def generate_dataset(n_states, n_features,n_samples, length, train_ratio, method, 
                     flip_probability= None, flip_probability_0=None, flip_probability_1=None,
                    startprob=None, transmat=None):
    
    #Generate Data
    dataset, states_true = generate_hmm(n_states, n_features , n_samples, length)
    
    x_train = dataset[:int(train_ratio*n_samples)]
    x_test = dataset[int(train_ratio*n_samples):]
    
   
    #Flip The Labels according to method
    
    states_flipped = []
    
    if method == "basic":
        for item in states_true:
            states_flipped.append(flip_labels_basic(item, flip_probability))
        
    elif method == "class":
        for item in states_true:
            states_flipped.append(flip_labels_class(item, flip_probability_0, flip_probability_1))
        
    elif method == "time":
        for item in states_true:
            states_flipped.append(flip_labels_time(item, startprob, transmat))
        
    y_train_true = states_true[:int(train_ratio*n_samples)]
    y_test_true = states_true[int(train_ratio*n_samples):]
    
    y_train_flipped = np.array(states_flipped[:int(train_ratio*n_samples)])
    y_test_flipped = np.array(states_flipped[int(train_ratio*n_samples):])
    
    
    return x_train, y_train_true, y_train_flipped, x_test, y_test_true, y_test_flipped

In [63]:
n_states = 2
n_features = 3
n_samples = 100
length = 1000
train_ratio = 0.7
method = "basic"
flip_probability = 0.1

x_train, y_train_true, y_train_flipped, x_test, y_test_true, y_test_flipped= generate_dataset(n_states, n_features,n_samples, length, train_ratio, method, 
                     flip_probability, flip_probability_0=None, flip_probability_1=None,
                    startprob=None, transmat=None)

In [64]:
x_train.shape

(70, 1000, 3)

In [65]:
n_states = 2
n_features = 3
n_samples = 100
length = 1000
train_ratio = 0.7
method = "class"
flip_probability_0 = 0.1
flip_probability_1 = 0.2

x_train, y_train_true, y_train_flipped, x_test, y_test_true, y_test_flipped = generate_dataset(n_states, n_features,n_samples, length, train_ratio, method, 
                     flip_probability= None, flip_probability_0=flip_probability_0, flip_probability_1 = flip_probability_1,
                    startprob=None, transmat=None)

In [66]:
x_train.shape

(70, 1000, 3)

In [67]:
n_states = 2
n_features = 3
n_samples = 100
length = 1000
train_ratio = 0.7
method = "time"

startprob = random_startprob(2)
transmat = np.array([[0.95, 0.05],
                    [0.95, 0.05]])

x_train, y_train_true, y_train_flipped, x_test, y_test_true, y_test_flipped = generate_dataset(n_states, n_features,n_samples, length, train_ratio, method, 
                     flip_probability=None, flip_probability_0=None, flip_probability_1=None,
                    startprob=startprob, transmat=transmat)

In [68]:
y_train_flipped.shape

(70, 1000)

In [88]:
train = data_utils.TensorDataset(torch.from_numpy(x_train).float(), torch.from_numpy(y_train_true))
train_loader = data_utils.DataLoader(train, batch_size=10, shuffle=True)

test = data_utils.TensorDataset(torch.from_numpy(x_test).float(), torch.from_numpy(y_test_true))
test_loader = data_utils.DataLoader(test, batch_size=25, shuffle=True)

In [82]:
train_flipped = data_utils.TensorDataset(torch.from_numpy(x_train).float(), torch.from_numpy(y_train_flipped))
train_fipped_loader = data_utils.DataLoader(train_flipped, batch_size=10, shuffle=True)

test_flipped = data_utils.TensorDataset(torch.from_numpy(x_test).float(), torch.from_numpy(y_test_flipped))
test_fipped_loader = data_utils.DataLoader(test_flipped, batch_size=25, shuffle=True)

In [78]:
def get_device():
    """Get a gpu if available."""
    if torch.cuda.device_count()>0:
        device = torch.device('cuda')
        print("Connected to a GPU")
    else:
        print("Using the CPU")
        device = torch.device('cpu')
    return device

def which_device(model):
    return next(model.parameters()).device


def add_channels(X):
    if len(X.shape) == 2:
        return X.reshape(X.shape[0], 1, X.shape[1],1)

    elif len(X.shape) == 3:
        return X.reshape(X.shape[0], 1, X.shape[1], X.shape[2])

    else:
        return "dimenional error"
    
def exp_lr_scheduler(epoch, optimizer, strategy='normal', decay_eff=0.1, decayEpoch=[]):
    """Decay learning rate by a factor of lr_decay every lr_decay_epoch epochs"""

    if strategy=='normal':
        if epoch in decayEpoch:
            for param_group in optimizer.param_groups:
                param_group['lr'] *= decay_eff
            print('New learning rate is: ', param_group['lr'])
    else:
        print('wrong strategy')
        raise ValueError('A very specific bad thing happened.')

    return optimizer

    
    
def gaussian_init_(n_units, std=1):    
    sampler = torch.distributions.Normal(torch.Tensor([0]), torch.Tensor([std/n_units]))
    A_init = sampler.sample((n_units, n_units))[..., 0]  
    return A_init

In [79]:
class NoisyRNN(nn.Module):
    def __init__(self, input_dim, output_classes, n_units=128, eps=0.01, 
                 beta=0.8, gamma_A=0.01, gamma_W=0.01, init_std=1, alpha=1,
                 solver='base', add_noise=0, mult_noise=0):
        super(NoisyRNN, self).__init__()

        self.device = get_device()


        self.n_units = n_units
        self.eps = eps
        self.solver = solver
        self.beta = beta
        self.alpha = alpha
        self.gamma_A = gamma_A
        self.gamma_W = gamma_W
        self.add_noise = add_noise
        self.mult_noise = mult_noise
        
        self.tanh = nn.Tanh()

        self.E = nn.Linear(input_dim, n_units)
        self.D = nn.Linear(n_units, output_classes)     
                                            
        self.C = nn.Parameter(gaussian_init_(n_units, std=init_std))            
        self.B = nn.Parameter(gaussian_init_(n_units, std=init_std))    
        self.I = torch.eye(n_units).to(self.device)   

        self.d = nn.Parameter(torch.rand(self.n_units).float().to(self.device)*0 + eps)           


    def forward(self, x, mode='test'):
        T = x.shape[1]
        h = torch.zeros(x.shape[0], self.n_units).to(which_device(self))

        for i in range(T):
            z = self.E(x[:,i,:])

            if i == 0:
                    A = self.beta * (self.B - self.B.transpose(1, 0)) + (1-self.beta) * (self.B + self.B.transpose(1, 0)) - self.gamma_A * self.I
                    W = self.beta * (self.C - self.C.transpose(1, 0)) + (1-self.beta) * (self.C + self.C.transpose(1, 0)) - self.gamma_W * self.I
                
                        
            add_noise = 0.0
            mult_noise = 1.0
            if mode == 'train':
                if self.add_noise > 0:
                    add_noise = self.add_noise * torch.randn(h.shape[0], h.shape[1]).float().to(self.device)
                            
                if self.mult_noise > 0:
                    #mult_noise = self.mult_noise * torch.randn(h.shape[0], h.shape[1]).float().to(self.device) + 1
                    mult_noise = self.mult_noise * torch.rand(h.shape[0], h.shape[1]).float().to(self.device) + (1-self.mult_noise)
                        

            if self.solver == 'base': 
                h_update = self.alpha * torch.matmul(h, A) + self.tanh(torch.matmul(h, W) + z)                
                h = h + self.eps * h_update
            elif self.solver == 'noisy':
                h_update = self.alpha * torch.matmul(h, A) + self.tanh(torch.matmul(h, W) + z)                
                h = h + self.d * mult_noise * h_update + add_noise                              
                 
                
        # Decoder 
        #----------
        out = self.D(h)
        return out

In [89]:
#code for the driver

parser = argparse.ArgumentParser(description='Human Activity Data')
#
parser.add_argument('-f')
#
parser.add_argument('--name', type=str, default='mnist', metavar='N', help='dataset')
#
parser.add_argument('--batch-size', type=int, default=128, metavar='N', help='input batch size for training (default: 128)')
#
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N', help='input batch size for testing (default: 1000)')
#
parser.add_argument('--epochs', type=int, default=100, metavar='N', help='number of epochs to train (default: 90)')
#
parser.add_argument('--lr', type=float, default=0.02, metavar='LR', help='learning rate (default: 0.1)')
#
parser.add_argument('--lr_decay', type=float, default=0.1, help='learning rate decay value (default: 0.1)')
#
parser.add_argument('--lr_decay_epoch', type=int, nargs='+', default=[30], help='decrease learning rate at these epochs.')
#
parser.add_argument('--wd', default=0.0, type=float, metavar='W', help='weight decay (default: 0.0)')
#
parser.add_argument('--gamma_W', default=0.001, type=float, metavar='W', help='diffiusion rate for W')
#
parser.add_argument('--gamma_A', default=0.001, type=float, metavar='W', help='diffiusion rate for A')
#
parser.add_argument('--beta', default=0.75, type=float, metavar='W', help='skew level')
#
parser.add_argument('--model', type=str, default='NoisyRNN', metavar='N', help='model name')
#
parser.add_argument('--solver', type=str, default='noisy', metavar='N', help='model name')
#
parser.add_argument('--n_units', type=int, default=64, metavar='S', help='number of hidden units')
#
parser.add_argument('--eps', default=0.1, type=float, metavar='W', help='time step for euler scheme')
#
parser.add_argument('--T', default=49, type=int, metavar='W', help='time steps')
#
parser.add_argument('--init_std', type=float, default=0.1, metavar='S', help='control of std for initilization')
#
parser.add_argument('--seed', type=int, default=1, metavar='S', help='random seed (default: 0)')
#
parser.add_argument('--gclip', type=int, default=0, metavar='S', help='gradient clipping')
#
parser.add_argument('--optimizer', type=str, default='Adam', metavar='N', help='optimizer')
#
parser.add_argument('--alpha', type=float, default=1, metavar='S', help='for ablation study')
#
parser.add_argument('--add_noise', type=float, default=0.0, metavar='S', help='level of additive noise')
#
parser.add_argument('--mult_noise', type=float, default=0.0, metavar='S', help='level of multiplicative noise')
#
args = parser.parse_args()

if not os.path.isdir(args.name + '_results'):
    os.mkdir(args.name + '_results')

#==============================================================================
# set random seed to reproduce the work
#==============================================================================
torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)

#==============================================================================
# get device
#==============================================================================
device = get_device()

#==============================================================================
# get dataset
#==============================================================================
  
model = NoisyRNN(input_dim=int(3), output_classes=1000, n_units=args.n_units, 
              eps=args.eps, beta=args.beta, gamma_A=args.gamma_A, gamma_W=args.gamma_W,
              init_std=args.init_std, alpha=args.alpha,  solver=args.solver, 
              add_noise=args.add_noise, mult_noise=args.mult_noise).to(device)

torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)        
noise = torch.randn(1,70,1000,3).float()


#==============================================================================
# set random seed to reproduce the work
#==============================================================================
torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)


#==============================================================================
# Model summary
#==============================================================================
print(model)    
print('**** Setup ****')
print('Total params: %.2fk' % (sum(p.numel() for p in model.parameters())/1000.0))
print('************')    
   

if args.optimizer == 'SGD':
    optimizer = torch.optim.SGD(model.parameters(), lr=args.lr, momentum=0.9, weight_decay=args.wd)
elif  args.optimizer == 'Adam':
    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.wd)
else:
    print("Unexpected optimizer!")
    raise 


loss_func = nn.CrossEntropyLoss().to(device)

# training and testing
count = 0
loss_hist = []
test_acc = []

t0 = timeit.default_timer()
for epoch in range(args.epochs):
    model.train()
    lossaccum = 0
    
    for step, (x, y) in enumerate(train_loader):
        count += 1
        
        # Reshape data for recurrent unit
        inputs = Variable(x.view(-1, 1000, int(3))).to(device) # reshape x to (batch, time_step, input_size)         
        targets = Variable(y).to(device)

                 
        # send data to recurrent unit    
        output = model(inputs, mode='train')
        loss = loss_func(output, targets.float())
        
        
        optimizer.zero_grad()
        loss.backward()          
        
        if args.gclip != 0.0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), args.gclip) # gradient clip
            
        optimizer.step() # update weights
        lossaccum += loss.item()

        if args.model == 'test':
            D = model.W.weight.data.cpu().numpy()  
            u, s, v = np.linalg.svd(D, 0)
            model.W.weight.data = torch.from_numpy(u.dot(v)).float().cuda()

    loss_hist.append(lossaccum)    
     
    if epoch % 1 == 0:
        model.eval()
        correct = 0
        total_num = 0
        for data, target in test_loader: 
            data, target = data.to(device), target.to(device)               
            output = model(data.view(-1, 1, int(3)))                  
            
            pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
            #print(output.shape)
            #print(pred.shape)
            correct += pred.eq(target.data.view_as(pred)).cpu().sum().item()
            total_num += len(data)
        
        accuracy = correct / total_num
        test_acc.append(accuracy)
        print('Epoch: ', epoch, 'Iteration: ', count, '| train loss: %.4f' % loss.item(), '| test accuracy: %.3f' % accuracy)


#        if args.model == 'NoisyRNN':
#            B = model.B.data.cpu().numpy()            
#            A = args.alpha * (args.beta * (B - B.T) + (1-args.beta) * (B + B.T) - args.gamma_A * np.eye(args.n_units))
#            A = 0.5 * (A + A.T)
#            e, _ = np.linalg.eig(A)
#            print('Eigenvalues of A (min and max): ', (np.min(np.abs(e)), np.max(np.abs(e))))
#            
#            C = model.C.data.cpu().numpy()            
#            W = args.beta * (C - C.T) + (1-args.beta) * (C + C.T) - args.gamma_W * np.eye(args.n_units)
#            e, _ = np.linalg.eig(W)
#            print('Eigenvalues of A (min and max): ', (np.min(np.abs(e)), np.max(np.abs(e))))
            
             

    # schedule learning rate decay    
    optimizer=exp_lr_scheduler(epoch, optimizer, decay_eff=args.lr_decay, decayEpoch=args.lr_decay_epoch)

print('total time: ', timeit.default_timer()  - t0 )


torch.save(model, args.name + '_results/' + args.model + '_' + args.name + '_T_' + str(args.T) 
            + '_units_' + str(args.n_units) + '_beta_' + str(args.beta) 
            + '_gamma_A_' + str(args.gamma_A) + '_gamma_W_' + str(args.gamma_W) + '_eps_' + str(args.eps) 
            + '_solver_' + str(args.solver) + '_gclip_' + str(args.gclip) + '_optimizer_' + str(args.optimizer)
            + '_addnoise_' + str(args.add_noise) + '_multnoise_' + str(args.mult_noise) 
            + '_seed_' + str(args.seed) + '.pkl')  

data = {'loss': lossaccum, 'testacc': test_acc}
f = open(args.name + '_results/' + args.model + '_' + args.name + '_T_' + str(args.T) 
            + '_units_' + str(args.n_units) + '_beta_' + str(args.beta) 
            + '_gamma_A_' + str(args.gamma_A) + '_gamma_W_' + str(args.gamma_W) + '_eps_' + str(args.eps) 
            + '_solver_' + str(args.solver) + '_gclip_' + str(args.gclip) + '_optimizer_' + str(args.optimizer)
            + '_addnoise_' + str(args.add_noise) + '_multnoise_' + str(args.mult_noise) 
            + '_seed_' + str(args.seed) + '_loss.pkl',"wb")

pickle.dump(data,f)
f.close()

Using the CPU
Using the CPU
NoisyRNN(
  (tanh): Tanh()
  (E): Linear(in_features=3, out_features=64, bias=True)
  (D): Linear(in_features=64, out_features=1000, bias=True)
)
**** Setup ****
Total params: 73.51k
************
Epoch:  0 Iteration:  7 | train loss: 16187.3438 | test accuracy: 0.000
Epoch:  1 Iteration:  14 | train loss: 32849.6680 | test accuracy: 0.000
Epoch:  2 Iteration:  21 | train loss: 10942.8926 | test accuracy: 0.000
Epoch:  3 Iteration:  28 | train loss: 6418.6597 | test accuracy: 0.000
Epoch:  4 Iteration:  35 | train loss: 6152.6553 | test accuracy: 0.000
Epoch:  5 Iteration:  42 | train loss: 40529.3984 | test accuracy: 0.000
Epoch:  6 Iteration:  49 | train loss: 8468.4795 | test accuracy: 0.000
Epoch:  7 Iteration:  56 | train loss: 6916.9517 | test accuracy: 0.000
Epoch:  8 Iteration:  63 | train loss: 6433.4702 | test accuracy: 0.000
Epoch:  9 Iteration:  70 | train loss: 6125.6685 | test accuracy: 0.000
Epoch:  10 Iteration:  77 | train loss: 5920.1216 | t